In [1]:
import os
import fitz  # PyMuPDF
import pandas as pd

# Dossier contenant les fichiers PDF
pdf_folder = 'pdf'

# Fonction pour extraire le texte d'un fichier PDF
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

# Liste des fichiers PDF dans le dossier
pdf_files = [f for f in os.listdir(pdf_folder) if f.endswith('.pdf')]

# Initialisation des listes pour le dataframe
texts = []
labels = []

# Parcours des fichiers PDF et extraction du texte et des labels
for pdf_file in pdf_files:
    pdf_path = os.path.join(pdf_folder, pdf_file)
    text = extract_text_from_pdf(pdf_path)
    texts.append(text)
    
    if 'BC' in pdf_file:
        labels.append('BC')
    elif 'Brochure' in pdf_file:
        labels.append('Brochure')
    elif 'Manuel d\'utilisation' in pdf_file or 'MU' in pdf_file:
        labels.append('MU')
    elif 'MM' in pdf_file:
        labels.append('MM')
    else:
        labels.append('Unknown')

# Création du dataframe
df = pd.DataFrame({'Text': texts, 'Label': labels})

In [2]:
df

,Text,Label
0,Informations client\nDate :\nLargeur hors tout...,BC
1,Prenez de la hauteur avec notre nouveau fauteu...,Brochure
2,\n \nInvacare® AVIVA™ FX MPS Maxx\nSystème él...,Unknown
3,Informations client\nDate :\nLargeur hors tout...,BC
4,Design. Technologie. Performance.\n Modulite\n...,Brochure
5,"Série Invacare® AVIVA™RX\nAVIVA RX20, AVIVA RX...",MM
6,\n \n \nSérie Invacare® AVIVA™ RX\nAVIVA RX20...,MU
7,Informations client\nLargeur hors tout\nà part...,BC
8,Design. Technologie. Performance.\n 40 Ultra L...,Brochure
9,Catalogue produits\nEdition 2025\nNotre histoi...,Unknown


In [3]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
import spacy
import re

# Télécharger les ressources nécessaires de nltk
'''
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
'''

# Charger le modèle de langue de spacy
try:
    nlp = spacy.load('fr_core_news_sm')
except OSError:
    from spacy.cli import download
    download('fr_core_news_sm')
    nlp = spacy.load('fr_core_news_sm')

# Initialiser les objets nécessaires
stop_words = set(stopwords.words('french'))
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

def preprocess_text(text):
    # Suppression des caractères spéciaux et des sauts de ligne
    text = re.sub(r'\W+', ' ', text)
    text = text.replace('\n', ' ')
    
    # Tokenisation
    tokens = nltk.word_tokenize(text)
    
    # Normalisation et suppression des stop words
    tokens = [token.lower() for token in tokens if token.isalpha() and token.lower() not in stop_words]
    
    # Lemmatisation
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    
    # Stemming
    tokens = [stemmer.stem(token) for token in tokens]
    
    return ' '.join(tokens)

# Appliquer le traitement de texte à chaque texte dans la liste
processed_texts = [preprocess_text(text) for text in texts]

# Mettre à jour le dataframe avec les textes traités
df['Processed_Text'] = processed_texts
df

,Text,Label,Processed_Text
0,Informations client\nDate :\nLargeur hors tout...,BC,inform client date largeur hor tout mm hauteur...
1,Prenez de la hauteur avec notre nouveau fauteu...,Brochure,prenez hauteur nouveau fauteuil verticalisateu...
2,\n \nInvacare® AVIVA™ FX MPS Maxx\nSystème él...,Unknown,invacar aviva fx mp maxx systèm électriqu modu...
3,Informations client\nDate :\nLargeur hors tout...,BC,inform client date largeur hor tout mm hauteur...
4,Design. Technologie. Performance.\n Modulite\n...,Brochure,design technologi perform modulit fauteuil rou...
5,"Série Invacare® AVIVA™RX\nAVIVA RX20, AVIVA RX...",MM,série invacar aviva rx aviva aviva aviva fr fa...
6,\n \n \nSérie Invacare® AVIVA™ RX\nAVIVA RX20...,MU,série invacar aviva rx aviva aviva aviva rx fr...
7,Informations client\nLargeur hors tout\nà part...,BC,inform client largeur hor tout partir mm date ...
8,Design. Technologie. Performance.\n 40 Ultra L...,Brochure,design technologi perform ultra low maxx faute...
9,Catalogue produits\nEdition 2025\nNotre histoi...,Unknown,catalogu produit edit histoir permobil a vu jo...


In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Initialiser le vectoriseur TF-IDF
vectorizer = TfidfVectorizer()

# Vectoriser les textes prétraités
tfidf_matrix = vectorizer.fit_transform(df['Processed_Text'])

def search_documents(query, top_n=5):
    # Prétraiter la requête de l'utilisateur
    processed_query = preprocess_text(query)
    
    # Vectoriser la requête
    query_vector = vectorizer.transform([processed_query])
    
    # Calculer la similarité de cosinus entre la requête et les documents
    cosine_similarities = cosine_similarity(query_vector, tfidf_matrix).flatten()
    
    # Obtenir les indices des documents les plus similaires
    related_docs_indices = cosine_similarities.argsort()[-top_n:][::-1]
    
    # Retourner les documents les plus similaires
    return df.iloc[related_docs_indices]

# Exemple de recherche
query = "manuel d'utilisation fauteuil roulant invacare avivia rx"
result = search_documents(query)
result

,Text,Label,Processed_Text
6,\n \n \nSérie Invacare® AVIVA™ RX\nAVIVA RX20...,MU,série invacar aviva rx aviva aviva aviva rx fr...
13,M3 Corpus\nManuel d’utilisation | Français\nBi...,MU,corpu manuel utilis françai bienvenu famil per...
15,M5 Corpus\nManuel d’utilisation | Français\nBi...,MU,corpu manuel utilis françai bienvenu famil per...
26,\n \n \n \n \n \n \nInvacare® TDX® SP2 Series...,MU,invacar tdx seri fr fauteuil roulant électriqu...
22,\n \n \n \n \nSérie Invacare® Storm®4\nStorm4...,MU,série invacar storm x plore fr fauteuil roulan...


In [5]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
import requests
import json

# Étape 1: Analyse de la requête
def analyze_query(query):
    processed_query = preprocess_text(query)
    return processed_query

# Étape 2: Classification du problème
# Entraîner un classificateur pour la classification du problème
classifier = make_pipeline(TfidfVectorizer(), MultinomialNB())
classifier.fit(df['Processed_Text'], df['Label'])

def classify_problem(query):
    processed_query = analyze_query(query)
    label = classifier.predict([processed_query])[0]
    return label

# Étape 3: Base de règles
def apply_rules(label):
    rules = {
        'BC': 'Règle pour BC',
        'Brochure': 'Règle pour Brochure',
        'MU': 'Règle pour Manuel d\'utilisation',
        'MM': 'Règle pour MM',
        'Unknown': 'Règle pour inconnu'
    }
    return rules.get(label, 'Règle par défaut')

# Étape 4: Recherche dans les PDF
def search_pdfs(query):
    result = search_documents(query)
    return result

# Étape 5: Filtrage et sélection de l'information
def filter_information(result):
    filtered_info = result['Text'].tolist()
    return filtered_info

# Étape 6: Génération d'une réponse détaillée => à modifier pour utiliser une IA générative et fournir une réponse plus précise
def generate_response(query, filtered_info):
    prompt = "Voici la question : " + query + "Voici les informations trouvées par rapport à la question :\n\n"
    for info in filtered_info:
        prompt += info + "\n\n"
    
    url = "http://localhost:11434/api/generate"

    headers = {
        "Content-Type": "application/json",
    }
    
    data = {
        "model": "mistral",
        "prompt": prompt,
        "stream": False,
    }
    
    response = requests.post(url, headers=headers, data=json.dumps(data))
    
    actual_response = response.text
    if response.status_code == 200:
        response_text = response.text
        data = json.loads(response_text)
        actual_response = data['response']
    
    return actual_response

# Étape 7: Réponse à l'utilisateur
def respond_to_user(query):
    label = classify_problem(query)
    rule = apply_rules(label)
    result = search_pdfs(query)
    filtered_info = filter_information(result)
    response = generate_response(query, filtered_info)
    return response

In [6]:
# Exemple d'utilisation
query = "mon fauteuil roulant invacare avivia rx est trop bas, comment le réajuster ?"
response = respond_to_user(query)
print(response)

 Voici les caractéristiques techniques et l'entretien du modèle Invacare® Storm4 Série 1529688-AC :

   - Marque : Invacare
   - Modèle : Storm4
   - Référence : 1529688-AC
   - Pays d'origine : Belgique & Luxembourg (Invacare nv, Autobaan 22, B-8210 Loppem, Tel: (32) (0)50 83 10 10, Fax: (32) (0)50 83 10 11, marketingbelgium@invacare.com, www.invacare.be), Canada (Invacare Canada LP, c/o Motion Concepts Canada, 84 Citation Drive Unit 11, Concord, Ontario L4K 3C1), France (Invacare Poirier SAS, Route de St Roch, F-37230 Fondettes, Tel: (33) (0)2 47 62 64 66, contactfr@invacare.com, www.invacare.fr), Suisse/Suisse/Svizzera (Invacare AG, Neuhofweg 51, CH-4147 Aesch BL, Tel: (41) (0)61 487 70 80, Fax: (41) (0)61 488 19 10, switzerland@invacare.com), Allemagne (Invacare GmbH, Am Achener Hof 8, D-88316 Isny), Royaume-Uni (Invacare UK Operations Limited, Unit 4, Pencoed Technology Park, Pencoed, Bridgend CF35 5AQ, UK)
   - Type : Fauteuil roulant à basculement ou inclinaison d'assise
   - Co

In [ ]:
#